# Explainability of Classification Task on Iris Dataset

## 1. Global Explainability by using SHAP

In [ ]:
import shap
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

# Load Dataset
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")

# Split Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

# Model Training
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# SHAP Explainer
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

# Global Interpretation
# Calculate mean SHAP values for each class
shap_mean_values = np.mean(np.abs(shap_values), axis=1)

target_class = 2
shap_values_class = shap_values[target_class]

# Calculate mean SHAP values for each feature
mean_shap_values = np.mean(shap_values_class, axis=0)
sorted_indices = np.argsort(mean_shap_values)[::-1]
sorted_features = X.columns[sorted_indices]
sorted_shap_values = mean_shap_values[sorted_indices]

# Plot the waterfall plot
shap.plots._waterfall.waterfall_legacy(
    expected_value=explainer.expected_value[target_class],
    shap_values=sorted_shap_values,
    feature_names=X.columns,
    show=False
)
plt.title(f"Global SHAP Waterfall Plot (Class {target_class})")
plt.show()


## 2. Comparison between Local Explainability on Different ML Model by using LIME

### 2.1 Random Forest

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from lime.lime_tabular import LimeTabularExplainer
import matplotlib.pyplot as plt

# Load Dataset
data = load_iris()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")

# Split Dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

# Model Training
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Model Evaluation
y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

# LIME Explainer
explainer = LimeTabularExplainer(
    training_data=X_train.values,           
    feature_names=X.columns.tolist(),       
    class_names=data.target_names.tolist(), 
    mode="classification"                   
)

# Local interpretation
instance_idx = 4 
instance = X_test.iloc[instance_idx].values
true_label = y_test.iloc[instance_idx]
predicted_label = model.predict([instance])[0]

print(f"True Label: {data.target_names[true_label]}")
print(f"Predicted Label: {data.target_names[predicted_label]}")

explanation = explainer.explain_instance(
    data_row=instance, 
    predict_fn=model.predict_proba,  
    num_features=4                   
)

explanation.show_in_notebook(show_table=True)
explanation.as_pyplot_figure()
plt.show()

for feature, weight in explanation.as_list():
    print(f"Feature: {feature}, Weight: {weight}")


### 2.2 SVM 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from lime.lime_tabular import LimeTabularExplainer
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target
feature_names = iris.feature_names
class_names = iris.target_names

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

svm_model = make_pipeline(StandardScaler(), SVC(probability=True, kernel='linear'))
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=class_names))

explainer = LimeTabularExplainer(X_train, 
                                 training_labels=y_train, 
                                 feature_names=feature_names,
                                 class_names=class_names, 
                                 verbose=True, 
                                 mode='classification')

idx = 4
exp = explainer.explain_instance(X_test[idx], svm_model.predict_proba, num_features=4)

print(f"True Label: {class_names[y_test[idx]]}")
print(f"Predicted Label: {class_names[y_pred[idx]]}")
exp.show_in_notebook(show_table=True)

exp.as_pyplot_figure()
plt.tight_layout()
plt.show()


### 2.3 Logistic Regression

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import lime
from lime.lime_tabular import LimeTabularExplainer
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

data = load_iris()
X = data.data
y = data.target
feature_names = data.feature_names
class_names = data.target_names

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = LogisticRegression(max_iter=200)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=class_names))

explainer = LimeTabularExplainer(
    X_train,
    feature_names=feature_names,
    class_names=class_names,
    discretize_continuous=True,
    mode="classification"
)

idx = 4
sample = X_test[idx].reshape(1, -1)
predicted_class = model.predict(sample)[0]
predicted_proba = model.predict_proba(sample)[0]
print(f"Predicted Label: {class_names[predicted_class]}")
print(f"Predicted Probability: {predicted_proba}")

exp = explainer.explain_instance(
    X_test[idx],
    model.predict_proba,
    num_features=4
)

print(f"True Label: {class_names[y_test[idx]]}")
print(f"Predicted Label: {class_names[y_pred[idx]]}")
exp.show_in_notebook()

exp.as_pyplot_figure()
plt.tight_layout()
plt.show()



### 2.4 KNN

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import lime
from lime.lime_tabular import LimeTabularExplainer

iris = datasets.load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred, target_names=class_names))

explainer = LimeTabularExplainer(
    training_data=X_train,
    mode="classification",
    training_labels=y_train,
    feature_names=iris.feature_names,
    class_names=iris.target_names,
    discretize_continuous=True
)

idx = 4
data_point = X_test[idx].reshape(1, -1)

explanation = explainer.explain_instance(data_point[0], knn.predict_proba)

print(f"True Label: {class_names[y_test[idx]]}")
print(f"Predicted Label: {class_names[y_pred[idx]]}")
explanation.show_in_notebook()

explanation.as_pyplot_figure()
plt.tight_layout()
plt.show()

### 2.5 Naive Bayes

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
import lime
from lime.lime_tabular import LimeTabularExplainer

iris = datasets.load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3)

nb = GaussianNB()
nb.fit(X_train, y_train)

y_pred = nb.predict(X_test)
print(classification_report(y_test, y_pred, target_names=class_names))

explainer = LimeTabularExplainer(
    training_data=X_train,
    mode="classification",
    training_labels=y_train,
    feature_names=iris.feature_names,
    class_names=iris.target_names,
    discretize_continuous=True
)

idx = 4 
data_point = X_test[idx].reshape(1, -1)

explanation = explainer.explain_instance(data_point[0], nb.predict_proba)

print(f"True Label: {class_names[y_test[idx]]}")
print(f"Predicted Label: {class_names[y_pred[idx]]}")
explanation.show_in_notebook()

explanation.as_pyplot_figure()
plt.tight_layout()
plt.show()


### 2.6 MLP

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import lime
from lime.lime_tabular import LimeTabularExplainer

iris = datasets.load_iris()
X = iris.data
y = iris.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

mlp = MLPClassifier(hidden_layer_sizes=(10,), max_iter=1000, random_state=42)
mlp.fit(X_train, y_train)

explainer = LimeTabularExplainer(
    training_data=X_train,
    mode="classification",
    training_labels=y_train,
    feature_names=iris.feature_names,
    class_names=iris.target_names,
    discretize_continuous=True
)

idx = 1
data_point = X_test[idx].reshape(1, -1)

explanation = explainer.explain_instance(data_point[0], mlp.predict_proba)

explanation.show_in_notebook()

explanation.as_pyplot_figure()
plt.tight_layout()
plt.show()
